In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.pipeline import make_pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, PolynomialFeatures, RobustScaler
from sklearn.ensemble import RandomForestClassifier


df_model = pd.read_csv('loan_project/SBAnational.csv')

/tmp/ipykernel_71687/283460630.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd
/tmp/ipykernel_71687/283460630.py:15: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  df_model = pd.read_csv('loan_project/SBAnational.csv')


In [2]:
df_model = df_model.dropna()
df_model = df_model.drop(['Name','CreateJob','RetainedJob','ChgOffDate', 'DisbursementDate','DisbursementGross','BalanceGross','ChgOffPrinGr','SBA_Appv'],axis=1)
df_model

,LoanNr_ChkDgt,City,State,Zip,Bank,BankState,NAICS,ApprovalDate,ApprovalFY,Term,NoEmp,NewExist,FranchiseCode,UrbanRural,RevLineCr,LowDoc,MIS_Status,GrAppv
6,1000093009,UNION,NJ,7083,WELLS FARGO BANK NATL ASSOC,SD,0,2-Jun-80,1980,45,45,2.0,0,0,N,N,CHGOFF,"$600,000.00"
36,1000554001,KINSMAN,OH,44428,CORTLAND SAVINGS & BK. CO.,OH,0,28-Feb-97,1997,137,2,1.0,1,0,N,Y,CHGOFF,"$47,000.00"
43,1000653000,EDINBURGH,IN,46124,JPMORGAN CHASE BANK NATL ASSOC,IN,0,11-Jun-80,1980,120,16,2.0,0,0,Y,N,CHGOFF,"$200,000.00"
58,1000726007,CLINTON (CENSUS NAME FOR CLINT,ME,4927,WELLS FARGO BANK NATL ASSOC,SD,236115,7-Feb-06,2006,167,9,1.0,1,1,0,N,CHGOFF,"$1,350,000.00"
60,1000735003,Spring Lake,MI,49417,WELLS FARGO BANK NATL ASSOC,SD,713930,31-Mar-97,1997,7,10,1.0,1,0,0,N,CHGOFF,"$25,000.00"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
899131,9995033006,CHATTANOOGA,TN,37416,FIRST VOLUNTEER BANK,TN,233210,27-Feb-97,1997,6,3,1.0,1,0,0,N,CHGOFF,"$125,000.00"
899138,9995143004,OVERLAND PARK,KS,66212,METCALF BANK,KS,0,27-Feb-97,1997,42,9,1.0,54100,0,N,Y,CHGOFF,"$100,000.00"
899143,9995213001,MARLTON,NJ,8053,BANCO POPULAR NORTH AMERICA,NJ,235610,27-Feb-97,1997,62,1,2.0,1,0,0,N,CHGOFF,"$150,000.00"
899153,9995413002,WOODBURY,NY,11797,FLUSHING BANK,NY,0,27-Feb-97,1997,119,5,1.0,1,0,0,N,CHGOFF,"$142,000.00"


In [3]:
df_model.isnull().sum()

LoanNr_ChkDgt    0
City             0
State            0
Zip              0
Bank             0
BankState        0
NAICS            0
ApprovalDate     0
ApprovalFY       0
Term             0
NoEmp            0
NewExist         0
FranchiseCode    0
UrbanRural       0
RevLineCr        0
LowDoc           0
MIS_Status       0
GrAppv           0
dtype: int64

In [4]:
colonne_categorie = ['LowDoc','RevLineCr']

tranformer = ColumnTransformer(
    transformers=[
        ('impute_colonne_c',SimpleImputer(strategy='constant', fill_value="Absent"),colonne_categorie)
        
    ],remainder='passthrough')

my_pipeline = make_pipeline(tranformer)

my_pipeline.fit(df_model)

df_model = my_pipeline.transform(df_model)

df_model = pd.DataFrame(df_model)
df_model

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
0,N,N,1000093009,UNION,NJ,7083,WELLS FARGO BANK NATL ASSOC,SD,0,2-Jun-80,1980,45,45,2.0,0,0,CHGOFF,"$600,000.00"
1,Y,N,1000554001,KINSMAN,OH,44428,CORTLAND SAVINGS & BK. CO.,OH,0,28-Feb-97,1997,137,2,1.0,1,0,CHGOFF,"$47,000.00"
2,N,Y,1000653000,EDINBURGH,IN,46124,JPMORGAN CHASE BANK NATL ASSOC,IN,0,11-Jun-80,1980,120,16,2.0,0,0,CHGOFF,"$200,000.00"
3,N,0,1000726007,CLINTON (CENSUS NAME FOR CLINT,ME,4927,WELLS FARGO BANK NATL ASSOC,SD,236115,7-Feb-06,2006,167,9,1.0,1,1,CHGOFF,"$1,350,000.00"
4,N,0,1000735003,Spring Lake,MI,49417,WELLS FARGO BANK NATL ASSOC,SD,713930,31-Mar-97,1997,7,10,1.0,1,0,CHGOFF,"$25,000.00"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
160866,N,0,9995033006,CHATTANOOGA,TN,37416,FIRST VOLUNTEER BANK,TN,233210,27-Feb-97,1997,6,3,1.0,1,0,CHGOFF,"$125,000.00"
160867,Y,N,9995143004,OVERLAND PARK,KS,66212,METCALF BANK,KS,0,27-Feb-97,1997,42,9,1.0,54100,0,CHGOFF,"$100,000.00"
160868,N,0,9995213001,MARLTON,NJ,8053,BANCO POPULAR NORTH AMERICA,NJ,235610,27-Feb-97,1997,62,1,2.0,1,0,CHGOFF,"$150,000.00"
160869,N,0,9995413002,WOODBURY,NY,11797,FLUSHING BANK,NY,0,27-Feb-97,1997,119,5,1.0,1,0,CHGOFF,"$142,000.00"


In [5]:
X = df_model.drop('MIS_Status', axis=1)
y = df_model.MIS_Status

X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=True, train_size=0.85, random_state=42)

num_col = list(X.select_dtypes(include=[float,int]).columns)
cat_col = list(X.select_dtypes(include=[object]).columns)

onehotscale_pipeline = make_pipeline(OneHotEncoder(), RobustScaler(with_centering=False))
scale_pipeline = make_pipeline(RobustScaler(with_centering=False))

preprocessing = ColumnTransformer(
    transformers=[
        ('categorical', onehotscale_pipeline, cat_col),
        ('numerical', scale_pipeline, num_col)]
)

polyscale_pipeline = make_pipeline(PolynomialFeatures(2))

my_final_pipeline = make_pipeline(preprocessing, polyscale_pipeline)
my_final_pipeline.fit(X_train)

feature_names = my_final_pipeline.get_feature_names_out(X.columns)

model = make_pipeline(
    my_final_pipeline,
    RandomForestClassifier(random_state=42, max_iter=100000)
)

model.fit(X_train, y_train)
y_pred = model.predict(X_test)


# Calcul de l'accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

KeyError: "['MIS_Status'] not found in axis"